# Epic remaining cost estimation based on team velocity

Cost estimation for all open (not Completed, Rejected) epics in the project. The model estimates only not completed stories that are assigned to epics in Jira. 
In this approach bugs are not estimated in sprints and they're affecting calculations only trough team velocity. More bugs - velocity is lower and the cost of the epic will be higher and vice versa.

In [ ]:
import pandas as pd
from jira import JIRA

jira = JIRA('https://kainos-evolve.atlassian.net')

#load all open epics 
jql = 'project=VXT and type=epic and status not in (Completed, Rejected)'

epicsRaw = jira.search_issues(jql)

epics = pd.DataFrame()
epics['version'] = ''
epics['key'] = ''
epics['type'] = ''
epics['status'] = ''
epics['summary'] = ''

#add epics to dataframe
for issue in epicsRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        epics = epics.append(
            {'version': fixVersion.name, 
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'summary': issue.fields.summary,
            }, ignore_index=True)
            
epics

Find all issues related to those epics

In [ ]:
epicKeys = epics['key'].tolist()
jql = '"Epic Link" in (' + ", ".join(epicKeys) + ')'
jql

In [ ]:
issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()

issues['epic'] = ''
issues['key'] = ''
issues['type'] = ''
issues['status'] = ''
issues['SP'] = 0
issues['summary'] = ''

for issue in issuesRaw:
    issues = issues.append(
        {
         'key': issue.key,
         'type': issue.fields.issuetype.name,
         'status': issue.fields.status.name,
         'SP': issue.fields.customfield_10005,
         'summary': issue.fields.summary,
         'team' : str(issue.fields.customfield_14200),
         'epic': issue.fields.customfield_10008
        }, ignore_index=True)

#only open issues are calculated
issues = issues.loc[~(issues['status'].isin(['Completed', 'Rejected']))]
issues.sort_values(["epic", 'type', 'status'], inplace=True)

#bugs are not required to be estimated
issues['emptySP'] = ((issues.type == 'Story') & issues.SP.isnull())
issues['notEmptySP'] = (~issues.emptySP)

issues

In [ ]:
import numpy as np
#in python we can treat True as 1 and False as 0 so simple sum suffice to calculate count of 
#estimated and not estimated issues in each epic
aggrIssues = issues.groupby(['epic']).agg({'emptySP':'sum','notEmptySP':'sum', 'SP': 'sum'})
aggrIssues['estimatedPerc'] = np.ceil(aggrIssues.notEmptySP / (aggrIssues.notEmptySP + aggrIssues.emptySP) * 100)
aggrIssues = aggrIssues.reset_index()

aggrIssues

<div class="alert alert-block alert-success">
Set max & min estimated velocity of the team and team sprint costs
</div>

In [ ]:
minVelocity = 10
avgVelocity = 15
maxVelocity = 20
#assuming that a team have 4 developers and each one salary is 1000 / week and we have 2W sprints
sprintCosts = 4 * 1000 * 2

In [ ]:
del aggrIssues['emptySP']
del aggrIssues['notEmptySP']

#if the epic's stories are not estimated
aggrIssues['minCost'] = 'Data not sufficient to estimate'
aggrIssues['avgCost'] = 'Data not sufficient to estimate'
aggrIssues['maxCost'] = 'Data not sufficient to estimate'

#only calculate costs for fully estimated epics
aggrIssues.loc[((aggrIssues['estimatedPerc'] == 100)), ['minCost']] = np.ceil(aggrIssues.SP / maxVelocity * sprintCosts)
aggrIssues.loc[((aggrIssues['estimatedPerc'] == 100)), ['avgCost']] = np.ceil(aggrIssues.SP / avgVelocity * sprintCosts)
aggrIssues.loc[((aggrIssues['estimatedPerc'] == 100)), ['maxCost']] = np.ceil(aggrIssues.SP / minVelocity * sprintCosts)

#aggrIssues

In [ ]:
#load epic descriptions from Jira
epicNames = []
for epicKey in aggrIssues['epic']:
    epic = jira.issue(epicKey)
    epicNames.append(epic.fields.summary)
    
epicNames = pd.Series(epicNames)
aggrIssues['summary'] = epicNames.values



    

In [ ]:
#change column order
cols = ['epic', 'summary', 'SP', 'estimatedPerc', 'minCost', 'avgCost', 'maxCost']
aggrIssues = aggrIssues[cols]

aggrIssues